In [1]:
from webscrapping.database.sql_queries import ValorantQueries
import pandas as pd

In [2]:
vq = ValorantQueries()
vq.set_match(43621)

Connected to database valorant successfully
[('teams',), ('roundevents',), ('players',), ('events',), ('series',), ('matches',), ('maps',), ('rounds',), ('roundeconomies',), ('roundlocations',), ('playermapinstance',), ('assists',)]


In [3]:
states = vq.get_initial_state()
events = vq.get_event_table()
states

,Match ID,Round ID,Round,Player ID,Player Name,Status,Team ID,Player Side,Remaining Creds,Loadout Value,...,Weapon Name,Weapon Price,Utility Value,Agent Name,Starting Side,Controller,Duelist,Initiator,Sentinel,Has Operator
0,43621,695324,1,1843,ade,alive,444,attack,0,600,...,Classic,0,600,Viper,defense,1,0,0,0,False
1,43621,695324,1,1075,Bazzi,alive,444,attack,0,800,...,Ghost,500,800,Sova,defense,0,0,1,0,False
2,43621,695324,1,1824,Fisker,alive,444,attack,0,800,...,Sheriff,800,800,Jett,defense,0,1,0,0,False
3,43621,695324,1,1877,frz,alive,53,defense,50,950,...,Ghost,500,950,Reyna,attack,0,1,0,0,False
4,43621,695324,1,3105,gtnziN,alive,53,defense,100,700,...,Ghost,500,700,Jett,attack,0,1,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,43621,695345,22,876,Munchkin,alive,444,defense,750,4600,...,Vandal,2900,4600,KAY/O,defense,0,0,1,0,False
216,43621,695345,22,2388,neth,alive,444,defense,100,2550,...,Stinger,1100,2550,Sage,defense,0,0,0,1,False
217,43621,695345,22,2895,Saadhak,alive,53,attack,2450,4700,...,Phantom,2900,4700,Sage,attack,0,0,0,1,False
218,43621,695345,22,2294,Sacy,alive,53,attack,1300,4600,...,Phantom,2900,4600,Sova,attack,0,0,1,0,False


In [12]:
mad = vq.generate_match_alive_dict()
chosen_round = 1
chosen_event = 2
alive_list = mad[chosen_round][chosen_event]
query = states[(states["Round"] == chosen_round)]
row_pot = []
for row_index in range(len(query)):
    row = states.iloc[row_index].copy()
    player_name = row["Player Name"]
    status = row["Status"]
    is_alive = alive_list[player_name]
    row["Status"] = "Alive" if is_alive == 1 else "Dead"
    row_pot.append(row)
g = pd.DataFrame(row_pot)
g = g[g["Status"] == "Alive"]
agg_dict = {"Remaining Creds": "sum", "Loadout Value": "sum", "Shield": "sum", "Weapon Price": "sum",
            "Controller": "sum", "Duelist": "sum", "Initiator": "sum", "Sentinel": "sum", "Has Operator": "sum"}
g.groupby(["Player Side"]).agg(agg_dict).reset_index()
#g.apply(lambda x: x[x["Status"] == "Alive"]).

,Player Side,Remaining Creds,Loadout Value,Shield,Weapon Price,Controller,Duelist,Initiator,Sentinel,Has Operator
0,attack,0,2200,50,500,1,0,1,1,0
1,defense,250,3950,50,1500,1,2,1,1,0


In [17]:
match_alive_dict[3][3]

{'ade': 1,
 'Bazzi': 1,
 'Fisker': 0,
 'frz': 1,
 'gtnziN': 1,
 'Munchkin': 0,
 'neth': 1,
 'Saadhak': 1,
 'Sacy': 1,
 'sutecas': 1}